# Implementação DQN com prioritized replay no ambiente Swift com manipulador panda.

Replay buffer <br>
DNN para representar o agente atual e uma DNN alvo com uma taxa de atualização menor.

In [10]:
import torch
import json
import pickle
import random
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from IPython.display import clear_output
from scipy.signal import convolve, gaussian
import os
import io
import base64
import time
import glob
from IPython.display import HTML

import data_panda as rbt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
%matplotlib inline


#j3 range -0.08 a 3.75  #j2 range -0.07 a -3. #j1 range -1.8 a 1.76

### Sistema observável e com medidas dos ângulos disponíveis


    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max
        0       Joint1                   -4.8                    4.8
        1       Joint2                    -Inf                    Inf
        2       Joint3                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        
    Actions:
        Type: Discrete(9)
        Num   Three actions for each joint
        0     decrement joint j
        1     increment joint j
        2     decrement join  j

        #j3 range 0.0 a 3.7
        #j2 range 0.0 a -3.
        #j1 range -1.7 a 1.7


        



In [11]:
state_shape = 3
env=rbt.Panda_RL()
agent=rbt.DQNAgent(state_shape, epsilon=0).to(device)
env.renderize=True #stop robot viewing

In [12]:
env.panda

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

### Main loop



Intializing with a given state

In [13]:
q_far=np.array([ 0., -0.8 ,  0. , -0.0698,  0.,  3.3825,  0.    ])
agent.play(env,"results_2",tmax=300,q0=q_far,model="last")

/home/taniel/anaconda3/envs/RTB/lib/python3.10/site-packages/roboticstoolbox/robot/Link.py:1041: FutureWarning: base kwarg is deprecated, use pose instead
  warn("base kwarg is deprecated, use pose instead", FutureWarning)


Final score:1447.5879826434784 in 145 steps
Status: Done Collided


In [14]:
env.reset()

array([-1.18, -2.13,  1.58])

In [15]:
env.step([0,-1,0])

(array([-1.18, -2.14,  1.58]), -20.707407789459324, False, ['', ''])

In [16]:
env.step_penalty=-10
env.mag=10
print(env.step([0,0,-1]))
print(env.fitness())
print(env.step([0,0,1]))
print(env.fitness())

(array([-1.18, -2.14,  1.57]), -25.55818158228904, False, ['', ''])
12.1894
(array([-1.18, -2.14,  1.58]), 5.558181582289041, False, ['', ''])
12.146899999999999


In [17]:
print(env.step([0,0,1]))
print(env.fitness())

(array([-1.18, -2.14,  1.59]), 5.5574735013165935, False, ['', ''])
12.104600000000001


In [20]:
agent.play(env,"results_10",tmax=300)

Final score:1511.1141206356806 in 299 steps
Status:  


In [9]:
agent.play(env,"results_9",tmax=300,model="other")


Final score:1550.5818767259925 in 299 steps
Status:  


In [264]:
agent.play(env,"results_6",tmax=300,model="last")

Final score:-806.9342406364051 in 299 steps
Status:  


**Let us record a video of trained agent**

In [72]:
# # Animate learned policy
# save_dir='./videos/'
# #env = make_env(env_name)
# generate_animation(env, agent, save_dir=save_dir)
# [filepath] = glob.glob(os.path.join(save_dir, '*.mp4'))

# display_animation(filepath)

In [73]:
#TODO

# Run this on another environment in OpenAI Gym
# Create a robotic environment with more actions
#